In [ ]:
#import urllib2
from io import StringIO
import os
import xmltodict
import pandas as pd
import geopandas as gpd
import windrose

from datetime import datetime 
import statsmodels.api as sm

from scipy.stats import linregress
import matplotlib.pyplot as plt
import numpy as np
import sys
import requests
import glob

from pyproj import CRS
from pyproj import Transformer
from shapely.geometry import Point

import rasterstats as rs
import rasterio

from rasterio.plot import show
from rasterio.mask import mask
from rasterio.plot import show_hist
from shapely.geometry import box
from rasterstats import zonal_stats
from windrose import WindroseAxes
#import pycrs
#import earthpy as et
#import earthpy.plot as ep

from scipy.optimize import curve_fit

from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

#import pymannkendall as mk
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 10

Replot the data to see how well the dumb filter worked.

In [ ]:
def plotlystuff(datasets, colnames, chrttypes=None, datatitles=None, chrttitle = 'well', colors=None,
                two_yaxes = False, axisdesig= None, axislabels=['Levels','Barometric Pressure'],opac=None):
    
    if chrttypes is None:
        chrttypes = ['lines']*len(datasets)

    if opac is None:
        opac = [0.8]*len(datasets)
        
    if datatitles is None:
        datatitles = colnames
    
    if axisdesig is None:
        axisdesig = ['y1']*len(datasets)
        
    if colors is None:
        if len(datasets) <= 5: 
            colors = ['#228B22','#F7DC6F','#FF1493','#663399','#FF0000']
        else:
            colors = []
            for i in range(len(datasets)):
                colors[i] = '#{:02x}{:02x}{:02x}'.format(r(), r(), r())
        
    r = lambda: random.randint(0,255)
    
    modetypes = ['markers','lines+markers','lines']
    datum = {}
    
    for i in range(len(datasets)):
        datum['d' + str(i)] = go.Scatter(
            x=datasets[i].index,
            y=datasets[i][colnames[i]],
            name = datatitles[i],
            line = dict(color = colors[i]),
            mode = chrttypes[i],
            opacity = opac[i],
            yaxis = axisdesig[i])
        
    data = list(datum.values())

    layout = dict(
        title=chrttitle,
        xaxis=dict(
            rangeslider=dict(
                visible = True
            ),
            type='date',
            tickformat='%Y-%m-%d %H:%M'
        ),
        yaxis=dict(title=axislabels[0],
                   titlefont=dict(color='#1f77b4'),
                   tickfont=dict(color='#1f77b4')),
    )
    
    
    if two_yaxes:
        layout['yaxis2'] = dict(title=axislabels[1], titlefont=dict(color='#ff7f0e'),
                    tickfont=dict(color='#ff7f0e'),
                    anchor='x',
                    overlaying='y',
                    side='right',
                    position=0.15)

    fig = dict(data=data, layout=layout)
    iplot(fig, filename = 'well')
    return 

# Read in File

In [ ]:
data_directory = "G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/Data_Downloads/Wellington/wellington_csv/"
file_name = "CSV_23861_Time_Series_131.dat"
file_path = os.path.join(data_directory, file_name)

# assign column names to the various columns in the dataset
# these are the standard CS time-series table columns
col_names = ['fileno', 'year', 'doy', 'hhmm', 'ss', 'Ux', 'Uy', 'Uz', 'T_SONIC', 'diag_sonic', 'CO2_density',
         'CO2_density_fast_tmpr','H2O_density','diag_irga','T_SONIC_corr','TA_1_1_1',
          'PA','CO2_sig_strgth','H2O_sig_strgth']

# Specify datatypes of the unsigned integers, otherwise they are inferred as float64
dtypes = {'year':np.ushort,'doy':np.ushort,'hhmm':np.ushort,
          'diag_sonic':np.ubyte, 'diag_irga':np.ubyte}

# Specify which columns to keep, this leaves out the first column
uscol = ['year', 'doy', 'hhmm', 'ss', 'Ux', 'Uy', 'Uz', 'T_SONIC',
       'diag_sonic', 'CO2_density', 'CO2_density_fast_tmpr', 'H2O_density',
       'diag_irga', 'T_SONIC_corr', 'TA_1_1_1', 'PA', 'CO2_sig_strgth',
       'H2O_sig_strgth']


df = pd.read_csv(test_file,names=col_names,dtype=dtypes,usecols=uscol)

# Create Datetime Index for the File

In [ ]:
def dateconv(x):
    """This function take an array that includes year, day of year, hour minute (no colon), 
    and second and converts it into a datetime
    
    x = 4x1 array with year, day of year, hour minute (no colon), and decimal seconds
    
    returns datetime    
    """
    
    # see https://strftime.org/
    fmt = "%Y %j %H%M %S %f"

    day = x[1]
    hhmm = x[2]
    
    # splits decimal and int into two numbers
    ms, s = math.modf(x[3])
    
    # convert all numbers into a string
    dtstr = f"{x[0]:0.0f} {x[1]:0.0f} {hhmm:4.0f} {s:2.0f} {ms*1e6:6.0f}"

    # deals with error that happens when hours, minutes, and seconds are 0
    if hhmm == 0:
        dtstr = f"{x[0]:0.0f} {x[1]:0.0f} {s:2.0f} {ms*1e6:6.0f}"
        fmt = "%Y %j %S %f"
        
    return datetime.strptime(dtstr,fmt)

x = [1900,171,812,30.0]

print(f"Example 1: {dateconv(x)}")

x = [2019,1,0,30.0]

print(f"Example 2: {dateconv(x)}")

x = [2019,1,0,0.0]

print(f"Example 3: {dateconv(x)}")

In [ ]:
# Get start and end dates of the dataset
sdate = dateconv(df.loc[df.index[0],['year','doy','hhmm','ss']])
s2date = dateconv(df.loc[df.index[1],['year','doy','hhmm','ss']])
edate = dateconv(df.loc[df.index[-1],['year','doy','hhmm','ss']])
dtrng = pd.date_range(start=sdate,end=edate,freq='0.05S')

# Make sure the frequencies and the length of the data match up, if not convert all time data into a datetime index (slow)
if len(dtrng) == len(df) and pd.to_timedelta(dtrng.freq)==(s2date-sdate):
    df['datetime'] = dtrng
else:
    df['datetime'] = df[['year','doy','hhmm','ss']].apply(lambda x: dateconv(x),1)
df = df.set_index('datetime')
df = df.drop(['year','doy','hhmm','ss'],axis=1)

# Filter Data with Poor Signal Strength

In [ ]:
df = df.where(df['H2O_sig_strgth']>0.7, np.nan)
df['H2O_sig_strgth'].plot()

Plot the raw wind data. Note: plotting is SLOW!

In [ ]:
fix, ax = plt.subplots(3,1,sharex=True)
df['Ux'].plot(ax=ax[0],color='blue')
df['Uy'].plot(ax=ax[1],color='red')
df['Uz'].plot(ax=ax[2],color='purple')

ax[0].set_ylabel('Ux (m/s)')
ax[1].set_ylabel('Uy (m/s)')
ax[2].set_ylabel('Uz (m/s)')

# Rough Filtering for Major Spikes

Currently, this filter removes major spikes with a really dumb limit of 100.  This should be replaced with a smarter filter that has a moving window.

In [ ]:
cols = ['Ux','Uy','Uz']
df_sub = df.loc[:, cols]

lim = np.abs(df_sub) < 100

# replace outliers with nan
df.loc[:, cols] = df_sub.where(lim, np.nan)

Replot the data to test how well the filter worked. Note: plotting is SLOW!

In [ ]:
fix, ax = plt.subplots(3,1,sharex=True)
df['Ux'].plot(ax=ax[0],color='blue')
df['Uy'].plot(ax=ax[1],color='red')
df['Uz'].plot(ax=ax[2],color='purple')

ax[0].set_ylabel('Ux (m/s)')
ax[1].set_ylabel('Uy (m/s)')
ax[2].set_ylabel('Uz (m/s)')

Replot the data in an interactive plot to test how well the filter worked. Note: plotting is SLOW!

In [ ]:
plotlystuff([df], ['Ux'], chrttypes=None, datatitles=None, chrttitle = 'Ux', 
            axislabels=['Ux m/s'],opac=None)

More dumb filters.  Do not run these unless you want to see why they are dumb.

In [ ]:
#cols = df.select_dtypes('number').columns  # limits to a (float), b (int) and e (timedelta)
#df_sub = df.loc[:, cols]

#dumb_filters = {'z-score':np.abs((df_sub - df_sub.mean()) / df_sub.std(ddof=0)) < 2.8,
#               'quantile':np.logical_or(df_sub < df_sub.quantile(0.99, numeric_only=False),
#                                        df_sub > df_sub.quantile(0.01, numeric_only=False)),
#                'iqr':np.abs((df_sub - df_sub.median()) / (df_sub.quantile(0.75, numeric_only=False) - df_sub.quantile(0.25, numeric_only=False))) < 2.22
#               }
## OPTION 1: z-score filter: z-score < 3
## OPTION 2: quantile filter: discard 1% upper / lower values
## OPTION 3: iqr filter: within 2.22 IQR (equiv. to z-score < 3)

## replace outliers with nan
#df.loc[:, cols] = df_sub.where(dumb_filters['z-score'], np.nan)

Here is a moving window filter stolen from my [transducer processing library](https://github.com/utah-geological-survey/loggerloader/blob/master/loggerloader/loader.py#L646-L693).

In [ ]:
def smoother(df, p, win=30, sd=3):
    """Remove outliers from a pandas dataframe column and fill with interpolated values.
    warning: this will fill all NaN values in the DataFrame with the interpolate function
    Args:
        df (pandas.core.frame.DataFrame):
            Pandas DataFrame of interest
        p (string):
            column in dataframe with outliers
        win (int):
            size of window in time steps (default 30)
        sd (int):
            number of standard deviations allowed (default 3)
    Returns:
        Pandas DataFrame with outliers removed
    """
    df1 = df
    df1.loc[:, 'dp' + p] = df1.loc[:, p].diff()
    df1.loc[:, 'ma' + p] = df1.loc[:, 'dp' + p].rolling(window=win, center=True).mean()
    df1.loc[:, 'mst' + p] = df1.loc[:, 'dp' + p].rolling(window=win, center=True).std()
    for i in df.index:
        try:
            if abs(df1.loc[i, 'dp' + p] - df1.loc[i, 'ma' + p]) >= abs(df1.loc[i, 'mst' + p] * sd):
                df.loc[i, p] = np.nan
            else:
                df.loc[i, p] = df.loc[i, p]
        except ValueError:
            try:
                if abs(df1.loc[i, 'dp' + p] - df1.loc[i, 'ma' + p]) >= abs(df1.loc[:, 'dp' + p].std() * sd):
                    df.loc[i, p] = np.nan
                else:
                    df.loc[i, p] = df.loc[i, p]
            except ValueError:
                df.loc[i, p] = df.loc[i, p]

    try:
        df1 = df1.drop(['dp' + p, 'ma' + p, 'mst' + p], axis=1)
    except(NameError, ValueError):
        pass
    del df1
    try:
        df = df.drop(['dp' + p, 'ma' + p, 'mst' + p], axis=1)
    except(NameError, ValueError):
        pass
    df = df.interpolate(method='time', limit=30)
    df = df[1:-1]
    return df

In [ ]:
df1 = smoother(df, 'Ux')
df1['Ux'].plot()

In [ ]:
plotlystuff([df1], ['Ux'], chrttypes=None, datatitles=None, chrttitle = 'Ux', 
            axislabels=['Ux m/s'],opac=None)

# Gap Filling

Gap filling should be done with proxy values.  Martin uses [R-EddyProc](https://github.com/bgctw/REddyProc) for this. Evaporative fraction could be a good proxy.

In [ ]:
df = df.interpolate(method='time')